In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from matplotlib import pyplot as plt

import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import numpy as np

import json
import glob
import random
import pickle
from tqdm import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch import nn
import torch.nn.functional as F
import spotPython.torch.netcore as netcore
from numpy import meshgrid, array, ravel

from src.eda import EDA
eda = EDA()

In [3]:
torch.manual_seed(40)
random.seed(40)
np.random.seed(40)

In [4]:
pio.templates.default = "plotly_white"
plotly.offline.init_notebook_mode(connected=True)

In [5]:
colors = ['#46039f','#9c179e','#d8576b','#fb9f3a']

class FashionCNN(nn.Module):
    def __init__(self, l1=64):
        super(FashionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, l1)
        self.fc2 = nn.Linear(l1, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
class Net_FashionMNIST(netcore.Net_Core):
    def __init__(self, l1, lr_mult, batch_size, epochs, k_folds, patience,
    optimizer, sgd_momentum):
        super(Net_FashionMNIST, self).__init__(
            lr_mult=lr_mult,
            batch_size=batch_size,
            epochs=epochs,
            k_folds=k_folds,
            patience=patience,
            optimizer=optimizer,
            sgd_momentum=sgd_momentum,
        )

        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, l1)
        self.fc2 = nn.Linear(l1, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))])

    trainset = torchvision.datasets.FashionMNIST(data_dir,
        download=True,
        train=True,
        transform=transform)
    testset = torchvision.datasets.FashionMNIST(data_dir,
        download=True,
        train=False,
        transform=transform)

    return trainset, testset

def train_fashion_mnist(config):
    net = FashionCNN(config["config.l1"]) 

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    # # loading data
    trainset, testset = load_data()
    
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=config["config.batch_size"], shuffle=True, num_workers=2
    )

    # defining the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["config.learning_rate"])

    for epoch in range(config["config.epochs"]):
        running_loss = 0.0
        epoch_steps = 0
        with tqdm(enumerate(trainloader, 0), total=len(trainloader), unit="batch") as epoch_iterator:
            for i, data in epoch_iterator:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                epoch_steps += 1

                if i % 1000 == 999:
                    average_loss = running_loss / epoch_steps
                    epoch_iterator.set_description(f"Epoch {epoch+1}, Loss: {average_loss:.3f}")
                    running_loss = 0.0
            

    print("Training finished.")
    return net

def test_accuracy(net):
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

def test_class_accuracy(net):
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2
    )

    correct_per_class = [0] * len(class_names)
    total_per_class = [0] * len(class_names)
    correct = 0
    total = 0

    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            
            for i in range(len(labels)):
                label = labels[i]
                prediction = predicted[i]
                if label == prediction:
                    correct_per_class[label] += 1
                total_per_class[label] += 1
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy_per_class = [correct / total * 100 if total > 0 else 0 for correct, total in zip(correct_per_class, total_per_class)]

    for i, accuracy in enumerate(accuracy_per_class):
        print('Accuracy of {}: {:.2f}%'.format(class_names[i], accuracy))
        
    accuracy = correct / total
    print('Overall Accuracy: {:.2f}%'.format(accuracy * 100))
    return accuracy_per_class, accuracy

eval_data_path = './evaluation_data/'

def experimentPathToDataframe(experiment_path):
    df_results = pd.DataFrame()
    for file in glob.iglob(experiment_path + '/**/result.json', recursive=True):
        print(f"Loading results from {experiment_path}...")

        df = pd.read_json(file,lines=True)
        data = json.loads(df.to_json(orient='records'))
        df = pd.json_normalize(data, max_level=1)
        df_results = pd.concat([df_results, df], ignore_index=True)

    print(f"----------------------------\nFound {len(df_results.trial_id.unique())} trials.")
    return df_results

def experimentPathToSpotTuner(experiment_path):
    with open(experiment_path, 'rb') as f:
        spot_tuner =  pickle.load(f)
    return spot_tuner

def spotTunerToDataFrame(spotTuner):
    df = pd.DataFrame(np.concatenate((spotTuner.X, spotTuner.y.reshape(-1, 1)), axis=1), columns=spotTuner.var_name + ["y"])
    df['l1'] = df['l1'].apply(lambda x: int(2**x))
    df['lr_mult'] = df['lr_mult'].apply(lambda x: x*1e-3)
    df['batch_size'] = df['batch_size'].apply(lambda x: int(2**x))
    df['epochs'] = df['epochs'].apply(lambda x: [5, 10, 15, 20][int(x)])
    df = df.rename(columns={'l1': 'config.l1', 
                            'lr_mult': 'config.learning_rate',
                            'batch_size': 'config.batch_size',
                            'epochs': 'config.epochs',
                            'y': 'mean_val_loss'})
    print(f"----------------------------\nFound {len(df)} trials.")
    return df

def bestConfig(df):
    df_tmp = df.groupby('trial_id').max('training_iteration').reset_index()
    df_tmp = df_tmp[df_tmp['mean_val_loss'] == df_tmp['mean_val_loss'].min()]
    best_config = df_tmp[['mean_val_loss', 'mean_accuracy', 'config.l1', 'config.batch_size', 'config.epochs', 'config.learning_rate']].to_dict(orient='records')[0]
    # for key, value in best_config.items():
    #     print(f'{key}: {value}') 
    return best_config

def bestConfigSpot(spotTuner):
    results = spotTuner.print_results(print_screen=False)
    best_config = {
        'mean_val_loss': None,
        'mean_accuracy': None,
        'config.l1': None,
        'config.batch_size': None,
        'config.epochs': None,
        'config.learning_rate': None,
    }
    for conf in results:
        if conf[0] == 'l1':
            best_config['config.l1'] = int(2**conf[1])
        elif conf[0] == 'batch_size':
            best_config['config.batch_size'] = int(2**conf[1])
        elif conf[0] == 'epochs':
            best_config['config.epochs'] = [5, 10, 15, 20][int(conf[1])]
        elif conf[0] == 'lr_mult':
            best_config['config.learning_rate'] = conf[1] * 1e-3
    return best_config

def repeatedTraining(config, iter, file_name):
    res = []
    for i in range(iter):
        torch.manual_seed(i)
        random.seed(i)
        np.random.seed(i)
        score = {'name': file_name}
        net = train_fashion_mnist(config)
        class_acc, acc = test_class_accuracy(net)
        for val, name in zip(class_acc,class_names):
            score[name] = val
        score['overall'] = acc
        res.append(score)
        df = pd.DataFrame(res)
        df.to_csv(f'./results/evaluation_data/{file_name}.csv')
    return df

In [13]:
## Load Data
df_e2 = pd.read_csv(f'{eval_data_path}/tuning_logs/e2.csv', index_col=0)
df_e2[['config.learning_rate']] = df_e2[['config.learning_rate']].astype(float)
idx = df_e2.groupby('trial_id')['training_iteration'].idxmax()
df_flat_e2 = df_e2.loc[idx,]

df_e3 = pd.read_csv(f'{eval_data_path}/tuning_logs/e3.csv', index_col=0)
df_e3[['config.learning_rate']] = df_e3[['config.learning_rate']].astype(float)
idx = df_e3.groupby('trial_id')['training_iteration'].idxmax()
df_flat_e3 = df_e3.loc[idx,]

df_e4 = pd.read_csv(f'{eval_data_path}/tuning_logs/e4-1.csv', index_col=0)
df_e4[['config.learning_rate']] = df_e4[['config.learning_rate']].astype(float)
idx = df_e4.groupby('trial_id')['training_iteration'].idxmax()
df_flat_e4 = df_e4.loc[idx,]

df_e5 = pd.read_csv(f'{eval_data_path}/tuning_logs/e5-1.csv', index_col=0)
df_e5[['config.learning_rate']] = df_e5[['config.learning_rate']].astype(float)
idx = df_e5.groupby('trial_id')['training_iteration'].idxmax()
df_flat_e5 = df_e5.loc[idx,]

df_e6 = pd.read_csv(f'{eval_data_path}/tuning_logs/e6-1.csv', index_col=0)
df_e6[['config.learning_rate']] = df_e6[['config.learning_rate']].astype(float)
idx = df_e6.groupby('trial_id')['training_iteration'].idxmax()
df_flat_e6 = df_e6.loc[idx,]

df_e6_log = pd.read_csv(f'{eval_data_path}/tuning_logs/e6.csv', index_col=0)
df_e6_log[['config.learning_rate']] = df_e6_log[['config.learning_rate']].astype(float)
idx = df_e6_log.groupby('trial_id')['training_iteration'].idxmax()
df_flat_e6_log = df_e6_log.loc[idx,]

df_e7 = pd.read_csv(f'{eval_data_path}/tuning_logs/e7-1.csv', index_col=0)
df_e7[['config.learning_rate']] = df_e7[['config.learning_rate']].astype(float)
idx = df_e7.groupby('trial_id')['training_iteration'].idxmax()
df_flat_e7 = df_e7.loc[idx,]

df_e7_log = pd.read_csv(f'{eval_data_path}/tuning_logs/e7.csv', index_col=0)
df_e7_log[['config.learning_rate']] = df_e7_log[['config.learning_rate']].astype(float)
idx = df_e7_log.groupby('trial_id')['training_iteration'].idxmax()
df_flat_e7_log = df_e7_log.loc[idx,]

spotTuner_e8 = experimentPathToSpotTuner("experiment_data/e8_spotPython/spot_runs_30-08-2023.pkl")
df_flat_e8 = pd.read_csv(f'{eval_data_path}/tuning_logs/e8.csv', index_col=0)

default_reps = pd.read_csv(f'{eval_data_path}/reps/default_reps.csv', index_col=0)
e6_reps = pd.read_csv(f'{eval_data_path}/reps/e6-1_reps.csv', index_col=0)
e7_reps = pd.read_csv(f'{eval_data_path}/reps/e7-1_reps.csv', index_col=0)
e8_reps = pd.read_csv(f'{eval_data_path}/reps/e8-1_reps.csv', index_col=0)

In [7]:
default_config = {
    'mean_val_loss': None,
    'mean_accuracy': None,
    'config.l1': 64,
    'config.batch_size': 64,
    'config.epochs': 15,
    'config.learning_rate': 0.001,
}

configs = [default_config,
           bestConfig(df_e6),
           bestConfig(df_e7),
           bestConfigSpot(spotTuner_e8)]

df = pd.DataFrame(configs)[['config.l1', 'config.batch_size', 'config.epochs', 'config.learning_rate']]
df.index = ['Default', 'Hyperband', 'Random Search', 'SMBO']
df['trials'] = ['-', f'{len(df_flat_e6)} (~6h)', f'{len(df_flat_e7)} (~6h)', f'{len(df_flat_e8)} (~6h)']
df = df.sort_values(by=['config.l1','config.learning_rate'])
df.columns = ['L1 Units', 'Batch Size', 'Epochs', 'Learning Rate', '# Trials']
#df.to_csv(f'{eval_data_path}/best_config.csv', index=True)

<div style="display: flex; justify-content: space-between; align-items: center; width: 100%;">
    <span style="color: #ff6900; font-size: 40px; font-weight: bold;">Modeling</span>
    <span style="text-align: right; font-size: 12px;">Isabel Janez | Judith Romer</span>
</div>
<p style="font-size: 18px;"><i>Use of Machine Learning or other Data Mining Algorithms</i></p><br>

<div style="display: flex; justify-content: space-between; align-items: center; width: 100%;">
    <span style="color: #ff6900; font-size: 35px; font-weight: bold;">Tuning Approaches</span>
    <span style="text-align: right; font-size: 12px;">Lena Hammerer | Luisa Ibele | Hanna Steinwender</span>
</div>

What different behaviors can be observed in the approaches?

<p style="color: #ff6900; font-weight: bold; font-size: 25px">Random Search</p>

randomly and repeatedly choosing hyperparameters to evaluate <sup>[1][2]</sup>
- model-free undirected search approach
- baseline to compare with other approaches in experiment setup
- preferable to Grid Search for high-dimensional spaces and heterogenous parameter importance

<img style="margin-left: auto; margin-right: auto" src="images/randomSearch_gridSearch.png" width="50%">

<p style="color: #ff6900; font-weight: bold; font-size: 25px">Hyperband Algorithm</p>

multi-armed bandit strategy that dynamically allocates resources to a set of random configurations and uses successive halving to stop poorly performing configurations <sup>[3][4]</sup>
- model-free undirected search approach
- employs early stopping criteria (e.g., validation loss improvement) during training 
- allocates more resources (e.g., training epochs) to promising configurations and terminates poorly performing ones early
- efficiently identify the best-performing configurations within a given budget, avoiding wasteful computation

<img style="margin-left: auto; margin-right: auto" src="images/hyperband.png" width="60%">

<p style="color: #ff6900; font-weight: bold; font-size: 25px">Surrogate Model Based Optimization (SMBO)</p>

using the Sequential Parameter Optimization Toolkit (SPOT) <sup>[1][5]</sup>
- model-based directed search
- information gathered during search to learn relationship between hyperparameters/performance
- constructs a surrogate and looks for its minimum
- computational effort can be shifted from real evaluations to evaluations of the surrogate
- understanding hyperparameter importance and interactions between several hyperparameters

<img style="margin-left: auto; margin-right: auto" src="images/smbo.png" width="60%">

<br>
<footer style="font-size: 10px;">
<p>[1] Bartz, E., Bartz-Beielstein, T., Zaefferer, M. and Mersmann, O., 2023. Hyperparameter Tuning for Machine and Deep Learning with R: A Practical Guide (p. 71-78). Springer Nature. <br>
[2] Bergstra, James, and Y. Bengio. “Random Search for Hyper-Parameter Optimization.” The Journal of Machine Learning Research 13 (March 2012): 281–305.<br>
[3] Li, Lisha, Kevin Jamieson, Giulia DeSalvo, Afshin Rostamizadeh, and Ameet Talwalkar. “Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization.” June 18, 2018.<br>
[4] Li, Lisha, Kevin Jamieson, Afshin Rostamizadeh, Katya Gonina, Moritz Hardt, Benjamin Recht, and Ameet Talwalkar. “Massively Parallel Hyperparameter Tuning,” February 15, 2018.<br>
[5] Bartz-Beielstein, Thomas. “PyTorch Hyperparameter Tuning - A Tutorial for SpotPython.” June 7, 2023.</p>

<div style="display: flex; justify-content: space-between; align-items: center; width: 100%;">
    <span style="color: #ff6900; font-size: 35px; font-weight: bold;">Experiment Setup</span>
    <span style="text-align: right; font-size: 12px;">Lena Hammerer | Luisa Ibele | Hanna Steinwender</span>
</div>
<p style="color: #ff6900; font-weight: bold; font-size: 25px">Hyperparameter Search Spaces</p>
<p style="color: #ff6900; font-weight: bold; font-size: 25px">Utilized Technologies</p>
<p style="color: #ff6900; font-weight: bold; font-size: 25px">Computational Resources</p>

<div style="display: flex; justify-content: space-between; align-items: center; width: 100%;">
    <span style="color: #ff6900; font-size: 40px; font-weight: bold;">Evaluation</span>
    <span style="text-align: right; font-size: 12px;">Lena Hammerer | Luisa Ibele | Hanna Steinwender</span>
</div>
<p style="font-size: 18px;"><i>Verification of Predictive Accuracy and Coverage of the Use Case</i></p>

<span style="color: #ff6900; font-size: 35px; font-weight: bold;">Tuning Results</span>
<p style="color: #ff6900; font-weight: bold; font-size: 25px;">Best Configurations</p>

- more _Units_,
- smaller _Batch Size_,
- not a lot of _Epochs_ needed since ...
- ... fairly high _Learning Rate_!

In [31]:
pd.read_csv(f'{eval_data_path}/best_config.csv', index_col=0)

,L1 Units,Batch Size,Epochs,Learning Rate,# Trials
Default,64,64,15,0.001000,-
Random Search,128,16,10,0.054035,87 (~6h)
Hyperband,128,16,15,0.098538,340 (~6h)
SMBO,256,32,10,0.091179,68 (~6h)


<p style="font-weight: bold; font-size: 20px;">#️ of Trials - Why does it differ so much?</p>

- Hyperband algorithm focuses resources on promising configurations by combining random sampling and early elimination of poor configurations. More combinations can be tested with the same amount of resources (time).

In [32]:
e7_grouped = df_e7.groupby('config.epochs')
df7 = e7_grouped.get_group(20)

e6_grouped = df_e6.groupby('config.epochs')
df6 = e6_grouped.get_group(20)

fig = make_subplots(rows=1, cols=2, subplot_titles=["Random Search", "Hyperband"])

for trial in df7['trial_id'].unique().tolist():
    df = df7[df7['trial_id'] == trial]
    fig.add_trace(go.Scatter(x=df['training_iteration'], y=df['mean_val_loss'], mode='lines'), row=1, col=1)
fig.update_xaxes(title_text="Epoch", row=1, col=1)
fig.update_yaxes(title_text="Loss", row=1, col=1)

for trial in df6['trial_id'].unique().tolist():
    df = df6[df6['trial_id'] == trial]
    fig.add_trace(go.Scatter(x=df['training_iteration'], y=df['mean_val_loss'], mode='lines'), row=1, col=2)
fig.update_xaxes(title_text="Epoch", row=1, col=2)
fig.update_yaxes(title_text="Loss", row=1, col=2)

fig.update_layout(title_text="Training Loss Comparison", showlegend=False, width=900, height=400)

fig.show()

<span style="color: #ff6900; font-size: 35px; font-weight: bold;">Hyperparameter Interdependencies</span>
<p style="color: #ff6900; font-weight: bold; font-size: 25px;">Overview Tested Configurations</p>

- **Hyperband** is biased due to early termination of unpromising trials. Trials starting out with high loss cannot reach their full potential as they are not trained for all the epochs.

In [15]:
fig = px.parallel_coordinates(df_flat_e6_log, color="mean_val_loss",
                              title='Hyperband Trial Configurations',
                              dimensions=['config.l1', 'config.batch_size', 'config.learning_rate',
                                          'config.epochs'], width=900, height=400)
fig.show()

<p style="color: #ff6900; font-weight: bold; font-size: 25px;">Overview Tested Configurations</p>

- **SMBO** makes interdependencies less visible here due to focus on finding improved configurations.

In [16]:
fig = px.parallel_coordinates(df_flat_e8, color="mean_val_loss",
                              title='SMBO Trial Configurations',
                              dimensions=['config.l1', 'config.batch_size', 'config.learning_rate',
                                          'config.epochs'], width=900, height=400)
fig.show()

<p style="color: #ff6900; font-weight: bold; font-size: 25px;">Overview Tested Configurations</p>

- **Random Search** provides a big picture perspective across the whole search space.

In [17]:
fig = px.parallel_coordinates(df_flat_e7_log, color="mean_val_loss",
                              title='Random Search Trial Configurations',
                              dimensions=['config.l1', 'config.batch_size', 'config.learning_rate',
                                          'config.epochs'], width=900, height=400)
fig.show()

<p style="color: #ff6900; font-weight: bold; font-size: 25px;">Units in Layer 1 vs. Batch Size</p>

- \# of _Units in Layer 1_ does have very little influence on results
- bigger _Batch Size_ with no other configuration changes requires more resources but does not help performance

In [36]:
# min = np.array(df_flat_e2['mean_val_loss']).min()
# max = np.array(df_flat_e2['mean_val_loss']).max()

fig = make_subplots(rows=1, cols=3, subplot_titles=["Random Search", "Random Search", "SMBO"], horizontal_spacing=0.085)

plot = go.Scatter(
    x=df_flat_e2['config.l1'], 
    y=df_flat_e2['config.batch_size'], 
    marker_color=df_flat_e2['mean_val_loss'], 
    mode='markers')
fig.add_trace(plot, row=1, col=1)

plot = go.Heatmap(
    z=[df_flat_e2[df_flat_e2['config.batch_size']==x]['mean_val_loss'].tolist() for x in df_flat_e2['config.batch_size'].unique()],
    x=df_flat_e2['config.l1'].unique(),
    y=df_flat_e2['config.batch_size'].unique(),
    #zsmooth='best',
    coloraxis='coloraxis'
)
fig.add_trace(plot, row=1, col=2)

def getHPIndex(name):
    map = [(i, x[0]) for i, x in enumerate(spotTuner_e8.print_importance(print_screen=False))]
    return [x[0] for x in map if x[1] == name][0]

# lower and upper
i = getHPIndex('l1')
j = getHPIndex('batch_size')
x = np.linspace(spotTuner_e8.lower[i], spotTuner_e8.upper[i], num=50)
y = np.linspace(spotTuner_e8.lower[j], spotTuner_e8.upper[j], num=50)
X, Y = meshgrid(x, y)
# Predict based on the optimized results
z0 = np.mean(np.array([spotTuner_e8.lower, spotTuner_e8.upper]), axis=0)
zz = array([spotTuner_e8.surrogate.predict(array([spotTuner_e8.chg(x, y, z0, i, j)])) for x, y in zip(ravel(X), ravel(Y))])
zs = zz[:, 0]
Z = zs.reshape(X.shape)
min_z = np.min(Z)
max_z = np.max(Z)
plot = go.Heatmap(
    z=Z,
    x=[2**val for val in x],
    y=[2**val for val in y],
    #zsmooth='best',
    coloraxis='coloraxis2'
)
fig.add_trace(plot, row=1, col=3)

fig.update_xaxes(
    title = 'L1 Units',
    tickmode = 'array',
    tickvals = df_flat_e2['config.l1'].unique(),
    row=1, col=1
)
fig.update_yaxes(
    title = 'Batch Size',
    tickmode = 'array',
    tickvals = df_flat_e2['config.batch_size'].unique(),
    row=1, col=1
)
fig.update_xaxes(
    tickmode = 'array',
    tickvals = df_flat_e2['config.l1'].unique(),
    row=1, col=2
)
fig.update_yaxes(
    tickmode = 'array',
    tickvals = df_flat_e2['config.batch_size'].unique(),
    row=1, col=2
)

fig.update_xaxes(
    tickmode = 'array',
    tickvals = df_flat_e2['config.l1'].unique(),
    row=1, col=3
)
fig.update_yaxes(
    tickmode = 'array',
    tickvals = df_flat_e2['config.batch_size'].unique(),
    row=1, col=3
)

fig.update_layout(coloraxis=dict(colorbar_x=0.64, colorbar_thickness=15),
                  coloraxis2=dict(colorbar_x=1, colorbar_thickness=15))

fig.update_layout(title='Mean Validation Loss: L1 vs. Batch Size', showlegend=False, width=900, height=400)
fig.show()

<p style="color: #ff6900; font-weight: bold; font-size: 25px;">Tested Configurations Overview</p>

- **Random Search** provides a big picture perspective across the whole search space.

In [18]:
fig = px.parallel_coordinates(df_flat_e7_log, color="mean_val_loss",
                              title='Random Search Trial Configurations',
                              dimensions=['config.l1', 'config.batch_size', 'config.learning_rate',
                                          'config.epochs'], width=900)
fig.show()

<p style="color: #ff6900; font-weight: bold; font-size: 25px;">Batch Size vs. Epochs</p>

- together determine number of training steps and therefore influence runtime
- large _Batch Size_ and few _Epochs_ does not work
- Choosing a smaller _Batch Size_ and training longer leads to better model performance

In [38]:
fig = make_subplots(rows=1, cols=3, subplot_titles=["Random Search", "Random Search", "SMBO"], horizontal_spacing=0.085)

plot = go.Scatter(
    x=df_flat_e3['config.batch_size'], 
    y=df_flat_e3['config.epochs'], 
    marker_color=df_flat_e3['mean_val_loss'], 
    mode='markers')
fig.add_trace(plot, row=1, col=1)

plot = go.Heatmap(
    z=[df_flat_e3[df_flat_e3['config.epochs']==x]['mean_val_loss'].tolist() for x in df_flat_e3['config.epochs'].unique()],
    x=df_flat_e3['config.batch_size'].unique(),
    y=df_flat_e3['config.epochs'].unique(),
    #zsmooth='best',
    coloraxis='coloraxis'
)
fig.add_trace(plot, row=1, col=2)

def getHPIndex(name):
    map = [(i, x[0]) for i, x in enumerate(spotTuner_e8.print_importance(print_screen=False))]
    return [x[0] for x in map if x[1] == name][0]

# lower and upper
i = getHPIndex('batch_size')
j = getHPIndex('epochs')
x = np.linspace(spotTuner_e8.lower[i], spotTuner_e8.upper[i], num=50)
y = np.linspace(spotTuner_e8.lower[j], spotTuner_e8.upper[j], num=50)
X, Y = meshgrid(x, y)
# Predict based on the optimized results
z0 = np.mean(np.array([spotTuner_e8.lower, spotTuner_e8.upper]), axis=0)
zz = array([spotTuner_e8.surrogate.predict(array([spotTuner_e8.chg(x, y, z0, i, j)])) for x, y in zip(ravel(X), ravel(Y))])
zs = zz[:, 0]
Z = zs.reshape(X.shape)
min_z = np.min(Z)
max_z = np.max(Z)
plot = go.Heatmap(
    z=Z,
    x=[2**val for val in x],
    y=[(val+1)*5 for val in y],
    #zsmooth='best',
    coloraxis='coloraxis2'
)
fig.add_trace(plot, row=1, col=3)

fig.update_xaxes(
    title = 'Batch Size',
    tickmode = 'array',
    tickvals = df_flat_e3['config.batch_size'].unique(),
    row=1, col=1
)
fig.update_yaxes(
    title = 'Epochs',
    tickmode = 'array',
    tickvals = df_flat_e3['config.epochs'].unique(),
    row=1, col=1
)
fig.update_xaxes(
    tickmode = 'array',
    tickvals = df_flat_e3['config.batch_size'].unique(),
    row=1, col=2
)
fig.update_yaxes(
    tickmode = 'array',
    tickvals = df_flat_e3['config.epochs'].unique(),
    row=1, col=2
)

fig.update_xaxes(
    tickmode = 'array',
    tickvals = df_flat_e3['config.batch_size'].unique(),
    row=1, col=3
)
fig.update_yaxes(
    tickmode = 'array',
    tickvals = df_flat_e3['config.epochs'].unique(),
    row=1, col=3
)

fig.update_layout(coloraxis=dict(colorbar_x=0.64, colorbar_thickness=15),
                  coloraxis2=dict(colorbar_x=1, colorbar_thickness=15))

fig.update_layout(title='Mean Validation Loss: Batch Size vs. Epochs', showlegend=False, width=900, height=400)
fig.show()

<p style="color: #ff6900; font-weight: bold; font-size: 25px;">Tested Configurations Overview</p>

- **Random Search** provides a big picture perspective across the whole search space.
- <span style="color: #ff6900;">... why are there so many small _Learning Rate_ configurations tested?</span>

In [20]:
fig = px.parallel_coordinates(df_flat_e7_log, color="mean_val_loss",
                              title='Random Search Trial Configurations - Loguniform',
                              dimensions=['config.l1', 'config.batch_size', 'config.learning_rate',
                                          'config.epochs'], width=900)
fig.show()

<p style="color: #ff6900; font-weight: bold; font-size: 25px;">Tested Configurations Overview</p>

- **Random Search** provides a big picture perspective across the whole search space.
- <span style="color: #ff6900;">... why are there so many small _Learning Rate_ configurations tested?</span>

In [21]:
fig = px.parallel_coordinates(df_flat_e7, color="mean_val_loss",
                              title='Random Search Trial Configurations - Uniform',
                              dimensions=['config.l1', 'config.batch_size', 'config.learning_rate',
                                          'config.epochs'], width=900)
fig.show()

<span style="font-weight: bold; font-size: 20px;">Learning Rate Sampling Approaches</span>

- Uniform distribution sampling explores a broad range of values while Loguniform focuses on a range of learning rates that span multiple orders of magnitude <br/>-> beneficial as it's common in DL that values on a logarithmic scale work well
- Uniform sampling helps to assess the sensitivity to specific learning rates, but is more resource-intensive compared to tuning with Loguniform

In [41]:
# Generate random samples from a uniform distribution
num_samples = 1000
uniform_samples = np.random.uniform(1e-4, 1e-1, num_samples)

# Generate random samples from a loguniform distribution
loguniform_samples = np.exp(np.random.uniform(np.log(1e-4), np.log(1e-1), num_samples))

# Create a combined dataset for plotting
data = {
    "Random Samples": np.concatenate([uniform_samples, loguniform_samples]),
    "Distribution": np.concatenate([["Uniform"] * num_samples, ["Loguniform"] * num_samples])
}

# Create a histogram using Plotly Express
fig = px.histogram(
    data_frame=data,
    x="Random Samples",
    color="Distribution",
    nbins=50,
    opacity=0.8,
    marginal="rug",
    color_discrete_sequence=[colors[1], colors[3]]
)

# Customize the layout
fig.update_layout(
    title="Uniform vs. Loguniform Distribution",
    xaxis_title="Random Values",
    yaxis_title="Frequency",
    width=900,
    height=400
)
fig.update_traces(marker_line_width=1,marker_line_color="black")
# Show the plot
fig.show()

<p style="color: #ff6900; font-weight: bold; font-size: 25px;">Learning Rate vs. Batch Size</p>

- larger _Batch Sizes_ work better with a higher _Learning Rate_
- very high _Learning Rates_ are not beneficial

In [14]:
min = np.array(df_flat_e5['config.learning_rate']).min()
max = np.array(df_flat_e5['config.learning_rate']).max()

fig = make_subplots(rows=1, cols=3, subplot_titles=["Random Search", "Random Search", "SMBO"], horizontal_spacing=0.085)

plot = go.Scatter(
    x=df_flat_e5['config.learning_rate'], 
    y=df_flat_e5['config.batch_size'], 
    marker_color=df_flat_e5['mean_val_loss'], 
    mode='markers')
fig.add_trace(plot, row=1, col=1)

z = pd.pivot_table(df_flat_e5, values=['mean_val_loss'], index=['config.batch_size'], columns=['config.learning_rate'], fill_value=None, dropna=False).to_numpy()
plot = go.Heatmap(
    z=z,
    x=sorted(df_flat_e5['config.learning_rate'].unique()),
    y=sorted(df_flat_e5['config.batch_size'].unique()),
    #zsmooth='best',
    connectgaps=True,
    coloraxis='coloraxis'
)
fig.add_trace(plot, row=1, col=2)

def getHPIndex(name):
    map = [(i, x[0]) for i, x in enumerate(spotTuner_e8.print_importance(print_screen=False))]
    return [x[0] for x in map if x[1] == name][0]

# lower and upper
i = getHPIndex('lr_mult')
j = getHPIndex('batch_size')
x = np.linspace(spotTuner_e8.lower[i], spotTuner_e8.upper[i], num=50)
y = np.linspace(spotTuner_e8.lower[j], spotTuner_e8.upper[j], num=50)
X, Y = meshgrid(x, y)
# Predict based on the optimized results
z0 = np.mean(np.array([spotTuner_e8.lower, spotTuner_e8.upper]), axis=0)
zz = array([spotTuner_e8.surrogate.predict(array([spotTuner_e8.chg(x, y, z0, i, j)])) for x, y in zip(ravel(X), ravel(Y))])
zs = zz[:, 0]
Z = zs.reshape(X.shape)
min_z = np.min(Z)
max_z = np.max(Z)
plot = go.Heatmap(
    z=Z,
    x=[val*1e-3 for val in x],
    y=[2**val for val in y],
    #zsmooth='best',
    coloraxis='coloraxis2'
)
fig.add_trace(plot, row=1, col=3)

fig.update_xaxes(
    title = 'Learning Rate',
    tickmode = 'array',
    tickvals = [round(min, 4), round(max, 4)],
    ticktext = [round(min, 4), round(max, 4)],
    row=1, col=1
)
fig.update_yaxes(
    title = 'Batch Size',
    tickmode = 'array',
    tickvals = df_flat_e5['config.batch_size'].unique(),
    row=1, col=1
)
fig.update_xaxes(
    tickmode = 'array',
    tickvals = [round(min, 5), round(max, 4)],
    ticktext = [round(min, 5), round(max, 4)],
    row=1, col=2
)
fig.update_yaxes(
    tickmode = 'array',
    tickvals = df_flat_e5['config.batch_size'].unique(),
    row=1, col=2
)

fig.update_xaxes(
    tickmode = 'array',
    tickvals = [1e-4, 1e-1],
    ticktext = [1e-4, 1e-1],
    row=1, col=3
)
fig.update_yaxes(
    tickmode = 'array',
    tickvals = df_flat_e5['config.batch_size'].unique(),
    row=1, col=3
)

fig.update_layout(coloraxis=dict(colorbar_x=0.64, colorbar_thickness=15),
                  coloraxis2=dict(colorbar_x=1, colorbar_thickness=15))

fig.update_layout(title='Mean Validation Loss: Learning Rate vs. Batch Size', showlegend=False, width=900, height=400)
fig.show()

<p style="color: #ff6900; font-weight: bold; font-size: 25px;">Tested Configurations Overview</p>

- **Random Search** provides a big picture perspective across the whole search space.

In [22]:
fig = px.parallel_coordinates(df_flat_e7, color="mean_val_loss",
                              title='Random Search Trial Configurations - Uniform',
                              dimensions=['config.l1', 'config.batch_size', 'config.learning_rate',
                                          'config.epochs'], width=900)
fig.show()

<p style="color: #ff6900; font-weight: bold; font-size: 25px;">Learning Rate vs. Epochs</p>

- training with a small _Learning Rate_ over more _Epochs_ leads to better results
- when training for less _Epochs_ a larger _Learning Rate_ can be selected
- five _Epochs_ seems to be not enough in general

In [43]:
min = np.array(df_flat_e4['config.learning_rate']).min()
max = np.array(df_flat_e4['config.learning_rate']).max()

fig = make_subplots(rows=1, cols=3, subplot_titles=["Random Search", "Random Search", "SMBO"], horizontal_spacing=0.085)

plot = go.Scatter(
    x=df_flat_e4['config.learning_rate'], 
    y=df_flat_e4['config.epochs'], 
    marker_color=df_flat_e4['mean_val_loss'], 
    mode='markers')
fig.add_trace(plot, row=1, col=1)

z = pd.pivot_table(df_flat_e4, 
        values=['mean_val_loss'], 
        index=['config.epochs'], 
        columns=['config.learning_rate'], fill_value=None, dropna=False).to_numpy()
plot = go.Heatmap(
    z=z,
    x=sorted(df_flat_e4['config.learning_rate'].unique()),
    y=sorted(df_flat_e4['config.epochs'].unique()),
    #zsmooth='best',
    connectgaps=True,
    coloraxis='coloraxis'
)
fig.add_trace(plot, row=1, col=2)

def getHPIndex(name):
    map = [(i, x[0]) for i, x in enumerate(spotTuner_e8.print_importance(print_screen=False))]
    return [x[0] for x in map if x[1] == name][0]

# lower and upper
i = getHPIndex('lr_mult')
j = getHPIndex('epochs')
x = np.linspace(spotTuner_e8.lower[i], spotTuner_e8.upper[i], num=50)
y = np.linspace(spotTuner_e8.lower[j], spotTuner_e8.upper[j], num=50)
X, Y = meshgrid(x, y)
# Predict based on the optimized results
z0 = np.mean(np.array([spotTuner_e8.lower, spotTuner_e8.upper]), axis=0)
zz = array([spotTuner_e8.surrogate.predict(array([spotTuner_e8.chg(x, y, z0, i, j)])) for x, y in zip(ravel(X), ravel(Y))])
zs = zz[:, 0]
Z = zs.reshape(X.shape)
min_z = np.min(Z)
max_z = np.max(Z)
plot = go.Heatmap(
    z=Z,
    x=[val*1e-3 for val in x],
    y=[(val+1)*5 for val in y],
    #zsmooth='best',
    coloraxis='coloraxis2'
)
fig.add_trace(plot, row=1, col=3)

fig.update_xaxes(
    title = 'Learning Rate',
    tickmode = 'array',
    tickvals = [round(min, 4), round(max, 4)],
    ticktext = [round(min, 4), round(max, 4)],
    row=1, col=1
)
fig.update_yaxes(
    title = 'Epochs',
    tickmode = 'array',
    tickvals = df_flat_e4['config.epochs'].unique(),
    row=1, col=1
)
fig.update_xaxes(
    tickmode = 'array',
    tickvals = [round(min, 5), round(max, 4)],
    ticktext = [round(min, 5), round(max, 4)],
    row=1, col=2
)
fig.update_yaxes(
    tickmode = 'array',
    tickvals = df_flat_e4['config.epochs'].unique(),
    row=1, col=2
)

fig.update_xaxes(
    tickmode = 'array',
    tickvals = [1e-4, 1e-1],
    ticktext = [1e-4, 1e-1],
    row=1, col=3
)
fig.update_yaxes(
    tickmode = 'array',
    tickvals = df_flat_e4['config.epochs'].unique(),
    row=1, col=3
)

fig.update_layout(coloraxis=dict(colorbar_x=0.64, colorbar_thickness=15),
                  coloraxis2=dict(colorbar_x=1, colorbar_thickness=15))

fig.update_layout(title='Mean Validation Loss: Learning Rate vs. Epochs', showlegend=False, width=900, height=400)
fig.show()

<p style="color: #ff6900; font-weight: bold; font-size: 25px;">Influence of Hyperparameters</p>

- SMBO calculated importance supports previous observations
- adjusting _Learning Rate_ and _Epochs_ is more significant than changing _Batch Size_ or the # of hidden _Units in the L1 Layer_

In [44]:
theta = np.power(10, spotTuner_e8.surrogate.theta)
imp = 100 * theta / np.max(theta)
idx = np.where(imp > 0.001)[0]
var_name = [spotTuner_e8.var_name[i] for i in idx]
comb = sorted(zip(var_name, imp), key = lambda t: t[1])
names = [x[0] for x in comb]
impo = [x[1] for x in comb]

fig = go.Figure([go.Bar(y=names, 
                        x=impo,         
                        marker_color=colors,
                        marker_line_color='black',
                        marker_line_width=1,
                        opacity=0.8,
                        orientation='h')])
fig.update_layout(title="SMBO Hyperparameter Importance",
                    width=900,
                    height=500)
fig.show()

<span style="color: #ff6900; font-size: 35px; font-weight: bold;">Performance Comparison</span>
<p style="color: #ff6900; font-weight: bold; font-size: 25px;">Overall Accuracy Score Improvement</p>

- achieving better overall accuracy than default configuration after hyperparameter tuning
- only small performance deviations between different tuning approaches and results <br/>-> expected 'cos also quite similar best configurations

Best Configurations

In [23]:
means = [default_reps.overall.median() * 100, e7_reps.overall.median() * 100, e6_reps.overall.median() * 100, e8_reps.overall.median() * 100]
df = pd.read_csv(f'{eval_data_path}/best_config.csv', index_col=0)
df["Overall Acc."] = means
df.style.format({
  'Overall Acc.': lambda val: f'{val:,.2f} %',
})

,L1 Units,Batch Size,Epochs,Learning Rate,# Trials,Overall Acc.
Default,64,64,15,0.001000,-,81.99 %
Random Search,128,16,10,0.054035,87 (~6h),91.73 %
Hyperband,128,16,15,0.098538,340 (~6h),90.73 %
SMBO,256,32,10,0.091179,68 (~6h),91.61 %


<p style="color: #ff6900; font-weight: bold; font-size: 25px;">Overall Accuracy Score Improvement</p>

- achieving better overall accuracy than default configuration after hyperparameter tuning
- only small performance deviations between different tuning approaches and results <br/>-> expected 'cos also quite similar best configurations

In [46]:
x_data = ['Default', 'Random Search', 'Hyperband', 'SMBO']
y_data = [default_reps['overall'] * 100, e7_reps['overall'] * 100, e6_reps['overall'] * 100, e8_reps['overall'] * 100]
# colors = ['#1b0c41','#781c6d','#cf4446','#fb9b06',]
colors = ['#46039f','#9c179e','#d8576b','#fb9f3a']

fig = go.Figure()
for xd, yd, cls in zip(x_data, y_data, colors):
        fig.add_trace(go.Box(
            y=yd,
            name=xd,
            boxpoints='all',
            jitter=0.5,
            whiskerwidth=0.2,
            fillcolor=cls,
            opacity=0.8,
            marker_size=2,
            marker_color='black',
            line_width=1)
        )
fig.update_layout(
    title='Overall Accuracy With Best Model Configuration',
    yaxis_title='Accuracy (%)',
    xaxis_title='Tuning Approach',
    showlegend=True,
    width=900,
    height=450
)
fig.show()

<p style="color: #ff6900; font-weight: bold; font-size: 25px;">Per Class Accuracy Score Improvement</p>

- default configuration struggles with some classes in particular (Pullover, Shirt, etc.)
- no single configuration is best in classifying all different types of fashion items

In [47]:
fig = go.Figure()
names = ['Default', 'Random Search', 'Hyperband', 'SMBO']
colors = ['#46039f','#9c179e','#d8576b','#fb9f3a']
for i, df in enumerate([default_reps, e7_reps, e6_reps, e8_reps]):
    fig.add_trace(go.Bar(
        x=class_names,
        y=[df.loc[:, name].mean() for name in class_names],
        name=names[i],
        marker_color=colors[i],
        marker_line_color='black',
        opacity=0.8
    ))

fig.update_layout(
    title='Class Accuracy With Best Model Configuration',
    xaxis_title='Image Class',
    yaxis_title='Accuracy (%)',
    barmode='group',
    width=900,
    height=450,
    showlegend=True,
)

fig.show()

<div style="display: flex; justify-content: space-between; align-items: center; width: 100%;">
    <span style="color: #ff6900; font-size: 40px; font-weight: bold;">Conclusion</span>
    <span style="text-align: right; font-size: 12px;">Lena Hammerer | Luisa Ibele | Hanna Steinwender</span>
</div>
<p style="font-size: 18px;"><i>Providing a Summary and Key Takeaways, as well as offering Insights and Recommendations for Future Work</i></p>

<ul>
    <li>Fazit: learning rate sehr empfindlich</li>
    <li>Smapling der Werte für Tuning</li>
    <li>Ausblick: Optimizer, Dropout nicht angeschaut etc.</li>
</ul>